In [4]:
import os
from pathlib import Path
import pandas as pd
from utils import find_project_root
import openpyxl

In [19]:
# Load the FCC data as a dataframe
root = find_project_root()
DATA_DIR = root / "data" / "raw" / "fcc"
FILE_PATH = DATA_DIR / "provider_summary_geotype.xlsx"
fcc_df = pd.read_excel(FILE_PATH)
fcc_df.head()

,geography_type,geography_id,geography_desc,data_type,provider_id,res_st_pct,bus_iv_pct
0,CBSA (MSA),10100,"Aberdeen, SD Micro Area",Fixed Broadband,130077,0.4272,0.0909
1,CBSA (MSA),10100,"Aberdeen, SD Micro Area",Fixed Broadband,130228,0.5655,0.5655
2,CBSA (MSA),10100,"Aberdeen, SD Micro Area",Fixed Broadband,130403,0.1233,0.1417
3,CBSA (MSA),10100,"Aberdeen, SD Micro Area",Fixed Broadband,130405,0.0040,0.0040
4,CBSA (MSA),10100,"Aberdeen, SD Micro Area",Fixed Broadband,130627,1.0000,1.0000


In [20]:
# Check how geography type = place is represented
fcc_df['geography_type'].value_counts()

geography_type
Census Place              405037
County                     59521
CBSA (MSA)                 21583
Congressional District     15502
Tribal                      7834
State                       4475
National                    2215
Name: count, dtype: int64

In [21]:
# Retain only census places
fcc_places_df = fcc_df[fcc_df['geography_type'] == 'Census Place'].reset_index(drop=True)
fcc_places_df.info()
fcc_places_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405037 entries, 0 to 405036
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   geography_type  405037 non-null  object 
 1   geography_id    405037 non-null  object 
 2   geography_desc  405037 non-null  object 
 3   data_type       405037 non-null  object 
 4   provider_id     405037 non-null  int64  
 5   res_st_pct      405037 non-null  float64
 6   bus_iv_pct      405037 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 21.6+ MB


,geography_type,geography_id,geography_desc,data_type,provider_id,res_st_pct,bus_iv_pct
0,Census Place,100100,"Abanda CDP, AL",Fixed Broadband,130403,0.0103,0.0103
1,Census Place,100100,"Abanda CDP, AL",Fixed Broadband,130627,1.0000,1.0000
2,Census Place,100100,"Abanda CDP, AL",Fixed Broadband,131425,0.0000,0.2165
3,Census Place,100100,"Abanda CDP, AL",Fixed Broadband,290111,1.0000,1.0000
4,Census Place,100100,"Abanda CDP, AL",Fixed Broadband,430076,1.0000,1.0000


In [28]:
# Round res_st_pct and bus_iv_pct to 2 decimal places
# Create a place level dataframe with the following features:
# total_unq_providers, total_unq_fixed_providers, total_unq_mobile_providers
# fixed_providers_with_more_than_75pct_resi_coverage, fixed_providers_with_more_than_75pct_bus_coverage
# mobile_providers_with_more_than_75pct_resi_coverage, mobile_providers_with_more_than_75pct_bus_coverage

# Round res_st_pct and bus_iv_pct to 2 decimal places
fcc_places_df['res_st_pct'] = fcc_places_df['res_st_pct'].round(2)
fcc_places_df['bus_iv_pct'] = fcc_places_df['bus_iv_pct'].round(2)

fcc_places_df.head()

,geography_type,geography_id,geography_desc,data_type,provider_id,res_st_pct,bus_iv_pct
0,Census Place,100100,"Abanda CDP, AL",Fixed Broadband,130403,0.01,0.01
1,Census Place,100100,"Abanda CDP, AL",Fixed Broadband,130627,1.00,1.00
2,Census Place,100100,"Abanda CDP, AL",Fixed Broadband,131425,0.00,0.22
3,Census Place,100100,"Abanda CDP, AL",Fixed Broadband,290111,1.00,1.00
4,Census Place,100100,"Abanda CDP, AL",Fixed Broadband,430076,1.00,1.00


In [34]:
# Create place level dataframe
grouping_cols = ['geography_id'] # ,'geography_desc'] # SO FUNNY same geography_id has different names for Fixed and Broadband
unq_providers = fcc_places_df.groupby(grouping_cols)['provider_id'].nunique().reset_index(name='total_unq_providers')
unq_fixed_providers = fcc_places_df[fcc_places_df['data_type'] == 'Fixed Broadband'].groupby(grouping_cols)['provider_id'].nunique().reset_index(name='total_unq_fixed_providers')
unq_mobile_providers = fcc_places_df[fcc_places_df['data_type'] == 'Mobile Broadband'].groupby(grouping_cols)['provider_id'].nunique().reset_index(name='total_unq_mobile_providers')
fixed_resi_75pct = fcc_places_df[(fcc_places_df['data_type'] == 'Fixed Broadband') & (fcc_places_df['res_st_pct'] >= 0.75)].groupby(grouping_cols)['provider_id'].nunique().reset_index(name='fixed_providers_with_more_than_75pct_resi_coverage')
fixed_bus_75pct = fcc_places_df[(fcc_places_df['data_type'] == 'Fixed Broadband') & (fcc_places_df['bus_iv_pct'] >= 0.75)].groupby(grouping_cols)['provider_id'].nunique().reset_index(name='fixed_providers_with_more_than_75pct_bus_coverage')
mobile_resi_75pct = fcc_places_df[(fcc_places_df['data_type'] == 'Mobile Broadband') & (fcc_places_df['res_st_pct'] >= 0.75)].groupby(grouping_cols)['provider_id'].nunique().reset_index(name='mobile_providers_with_more_than_75pct_resi_coverage')
mobile_bus_75pct = fcc_places_df[(fcc_places_df['data_type'] == 'Mobile Broadband') & (fcc_places_df['bus_iv_pct'] >= 0.75)].groupby(grouping_cols)['provider_id'].nunique().reset_index(name='mobile_providers_with_more_than_75pct_bus_coverage')

# Merge all dataframes to create place level dataframe
place_level_df = unq_providers \
    .merge(unq_fixed_providers, on=grouping_cols, how='left') \
    .merge(unq_mobile_providers, on=grouping_cols, how='left') \
    .merge(fixed_resi_75pct, on=grouping_cols, how='left') \
    .merge(fixed_bus_75pct, on=grouping_cols, how='left') \
    .merge(mobile_resi_75pct, on=grouping_cols, how='left') \
    .merge(mobile_bus_75pct, on=grouping_cols, how='left')

# Fill NaN values with 0
place_level_df.fillna(0, inplace=True)

# Convert float columns to int
for col in place_level_df.columns:
    if place_level_df[col].dtype == 'float64':
        place_level_df[col] = place_level_df[col].astype(int) 
place_level_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32175 entries, 0 to 32174
Data columns (total 8 columns):
 #   Column                                               Non-Null Count  Dtype
---  ------                                               --------------  -----
 0   geography_id                                         32175 non-null  int64
 1   total_unq_providers                                  32175 non-null  int64
 2   total_unq_fixed_providers                            32175 non-null  int64
 3   total_unq_mobile_providers                           32175 non-null  int64
 4   fixed_providers_with_more_than_75pct_resi_coverage   32175 non-null  int64
 5   fixed_providers_with_more_than_75pct_bus_coverage    32175 non-null  int64
 6   mobile_providers_with_more_than_75pct_resi_coverage  32175 non-null  int64
 7   mobile_providers_with_more_than_75pct_bus_coverage   32175 non-null  int64
dtypes: int64(8)
memory usage: 2.0 MB


In [36]:
# Examine the dataframe
# place_level_df.head()

# Save the place level dataframe as a csv file
OUTPUT_DIR = find_project_root() / "data" / "intermediate" / "fcc"
place_level_df.to_csv(OUTPUT_DIR / "fcc_provider_place_level_summary.csv", index=False)